## 1.重新整理所有结果存入csv文件中，并按照index排序

In [19]:
import pandas as pd
import json
import os
import glob


# cross_session, cross_subject
scenario = "cross_subject"
# BNCI2014_001, BNCI2015_001, Pan2023, Shin2017A
dataset = "BNCI2014_001"

# 设置读取文件路径
base_path = os.path.join(scenario, dataset)
json_pattern = os.path.join(base_path, "result_sub*.json")

# 设置保存CSV结果文件路径
csv_result_dir = os.path.join(base_path, "csv_result")
os.makedirs(csv_result_dir, exist_ok=True)

# 初始化一个字典来存储不同被试的数据
subject_data = {}

# 初始化一个 DataFrame 用于保存所有被试的结果
all_data = pd.DataFrame()

# 遍历所有的 JSON 文件
for json_file in glob.glob(json_pattern):
    subject_id = os.path.basename(json_file).split('sub')[1].split('.json')[0]  # 获取被试编号
    with open(json_file, 'r') as f:
        for line in f:
            result = json.loads(line)
            if subject_id not in subject_data:
                subject_data[subject_id] = []
            subject_data[subject_id].append(result)

# 确定所有 index 范围
all_indices = sorted(set(index for subject in subject_data.values() for index in [res["index"] for res in subject]))

# 将每个被试的数据保存为 CSV 文件
for subject_id, data in subject_data.items():
    df = pd.DataFrame(data)
    
    # 确保所有 index 存在，添加缺失行
    missing_rows = []
    for index in all_indices:
        if index not in df['index'].values:
            missing_rows.append({"index": index})
    
    df_sorted = df.sort_values(by='index')

    # 重新排列列的顺序
    columns_order = [
        "index", "algorithm_id", "algorithm",
        "test_score", "train_time", "test_time"
    ] + [col for col in df_sorted.columns if col not in [
        "index", "algorithm_id", "algorithm", 
        "test_score", "train_time","test_time"
        ]]

    df_sorted = df_sorted[columns_order]

    # 保存为 CSV 文件
    csv_file_path = os.path.join(csv_result_dir, f"result_sub{subject_id}.csv")
    df_sorted.to_csv(csv_file_path, index=False)
    
    print(f"Subject {subject_id} results saved to {csv_file_path}")
    
    # 将数据添加到所有被试数据的 DataFrame 中
    all_data = pd.concat([all_data, df_sorted], ignore_index=True)

# 计算平均值
mean_data = all_data.groupby('index').agg(
    algorithm_id=('algorithm_id', 'first'),
    algorithm=('algorithm', 'first'),
    cs_method=('cs_method', 'first'),
    test_score=('test_score', 'mean'),
    test_score_std=('test_score', 'std'),
    train_time=('train_time', 'mean'),
    train_time_std=('train_time', 'std'),
    test_time=('test_time', 'mean'),
    test_time_std=('test_time', 'std'),
    N=('test_score', lambda x: x.notna().sum())  # 计算非缺失值的数量
).reset_index()

# 重新排列列的顺序
mean_columns_order = [
    "index", "algorithm_id", "algorithm", 
    "cs_method",
    "N",
    "test_score", "test_score_std",
    "train_time", "train_time_std",
    "test_time", "test_time_std"
]

mean_data = mean_data[mean_columns_order]

# 保存为结果均值 CSV 文件
result_mean_path = os.path.join(csv_result_dir, "result_mean.csv")
mean_data.to_csv(result_mean_path, index=False)   

print(f"Results mean saved to {result_mean_path}")

Subject 1 results saved to cross_subject\BNCI2014_001\csv_result\result_sub1.csv
Subject 2 results saved to cross_subject\BNCI2014_001\csv_result\result_sub2.csv
Subject 3 results saved to cross_subject\BNCI2014_001\csv_result\result_sub3.csv
Subject 4 results saved to cross_subject\BNCI2014_001\csv_result\result_sub4.csv
Subject 5 results saved to cross_subject\BNCI2014_001\csv_result\result_sub5.csv
Subject 6 results saved to cross_subject\BNCI2014_001\csv_result\result_sub6.csv
Subject 7 results saved to cross_subject\BNCI2014_001\csv_result\result_sub7.csv
Subject 8 results saved to cross_subject\BNCI2014_001\csv_result\result_sub8.csv
Subject 9 results saved to cross_subject\BNCI2014_001\csv_result\result_sub9.csv
Results mean saved to cross_subject\BNCI2014_001\csv_result\result_mean.csv
